In [1]:
import tensorflow as tf

In [2]:
tf.enable_eager_execution()
tfe = tf.contrib.eager


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
learning_rate = 0.001
num_steps = 1000
batch_size = 128
display_step = 100

n_hidden_1 = 256
n_hidden_2 = 256
num_input = 784
num_classes = 10

In [5]:
dataset = tf.data.Dataset.from_tensor_slices((mnist.train.images,mnist.train.labels))
dataset = dataset.repeat().batch(batch_size).prefetch(batch_size)
dataset_iter = tfe.Iterator(dataset)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
class NeuralNet(tfe.Network):
    def __init__(self):
        super(NeuralNet,self).__init__()
        self.layer1 = self.track_layer(tf.layers.Dense(n_hidden_1,activation=tf.nn.relu))
        self.layer2 = self.track_layer(tf.layers.Dense(n_hidden_2,activation=tf.nn.relu))
        self.out_layer = self.track_layer(tf.layers.Dense(num_classes))
        
    def call(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        return self.out_layer(x)
    
neural_net = NeuralNet()

Instructions for updating:
Please inherit from `tf.keras.Model`, and see its documentation for details. `tf.keras.Model` should be a drop-in replacement for `tfe.Network` in most cases, but note that `track_layer` is no longer necessary or supported. Instead, `Layer` instances are tracked on attribute assignment (see the section of `tf.keras.Model`'s documentation on subclassing). Since the output of `track_layer` is often assigned to an attribute anyway, most code can be ported by simply removing the `track_layer` calls.

`tf.keras.Model` works with all TensorFlow `Layer` instances, including those from `tf.layers`, but switching to the `tf.keras.layers` versions along with the migration to `tf.keras.Model` is recommended, since it will preserve variable names. Feel free to import it with an alias to avoid excess typing :).

Please inherit from `tf.keras.Model`, and see its documentation for details. `tf.keras.Model` should be a drop-in replacement for `tfe.Network` in most cases, but

In [7]:
def loss_fn(inference_fn,inputs,labels):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                             logits=inference_fn(inputs),labels=labels))
def accuracy_fn(inference_fn,inputs,labels):
    prediction = tf.nn.softmax(inference_fn(inputs))
    correct_pred = tf.equal(tf.argmax(prediction,1),labels)
    return tf.reduce_mean(tf.cast(correct_pred,tf.float32))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
grad = tfe.implicit_gradients(loss_fn)

In [8]:
average_loss = 0.
average_acc = 0.

for step in range(num_steps):
    d = dataset_iter.next()
    x_batch = d[0]
    y_batch = tf.cast(d[1],tf.int64)
    
    batch_loss = loss_fn(neural_net,x_batch,y_batch)
    average_loss += batch_loss
    
    batch_accuracy = accuracy_fn(neural_net,x_batch,y_batch)
    average_acc += batch_accuracy
    
    if step == 0:
        print("Initial loss={:.9f}".format(average_loss))
    
    optimizer.apply_gradients(grad(neural_net,x_batch,y_batch))
    
    if (step + 1) % display_step == 0 or step == 0:
        if step > 0:
            average_loss /= display_step
            average_acc /= display_step
        print("Step:", '%04d' % (step + 1), " loss=",
              "{:.9f}".format(average_loss), " accuracy=",
              "{:.4f}".format(average_acc))
        average_loss = 0.
        average_acc = 0.

Initial loss=2.368701935
Step: 0001  loss= 2.368701935  accuracy= 0.1016
Step: 0100  loss= 0.574926794  accuracy= 0.8311
Step: 0200  loss= 0.245345071  accuracy= 0.9287
Step: 0300  loss= 0.213926390  accuracy= 0.9380
Step: 0400  loss= 0.179341301  accuracy= 0.9470
Step: 0500  loss= 0.138849780  accuracy= 0.9595
Step: 0600  loss= 0.117794424  accuracy= 0.9646
Step: 0700  loss= 0.115002319  accuracy= 0.9665
Step: 0800  loss= 0.106258400  accuracy= 0.9680
Step: 0900  loss= 0.082878150  accuracy= 0.9752
Step: 1000  loss= 0.080152862  accuracy= 0.9761


In [9]:
testX = mnist.test.images
testY = mnist.test.labels

test_acc = accuracy_fn(neural_net,testX,testY)
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 0.9701
